In [1]:
%load_ext autoreload
%autoreload 2
import itertools
import os
import numpy as np
from collections import defaultdict, Counter
import operator
import pandas as pd
import json
import math
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS


%store -r resultDict
%store -r qidInfo

In [27]:
%store -r query2classic
%store -r query2best

for query, best in query2best.items():
    print(query, best, query2classic[query])
    
    
best_val = [y for x,y in query2best.values()]
classic_val= [y for x,y in query2classic.values()]

print(np.mean(best_val))
print(np.mean(classic_val))
print(len(query2best))

25 (210314, 0.929204) (111111, 0.902655)
103 (100111, 0.568966) (111111, 0.568966)
171 (111111, 0.439024) (111111, 0.439024)
15 (521313, 0.709497) (111111, 0.653631)
177 (111, 0.727273) (111111, 0.704545)
18 (101111, 0.607143) (111111, 0.607143)
112 (0, 0.0) (111111, 0.0)
62 (101011, 0.650943) (111111, 0.632075)
37 (10100, 1.0) (111111, 0.666667)
198 (212003, 0.741379) (111111, 0.62069)
134 (100001, 0.434783) (111111, 0.26087)
68 (130202, 0.577778) (111111, 0.377778)
163 (3005, 0.517857) (111111, 0.223214)
193 (111001, 0.387097) (111111, 0.333333)
138 (130000, 0.666667) (111111, 0.4)
29 (101021, 0.571429) (111111, 0.428571)
34 (211213, 0.421687) (111111, 0.313253)
164 (213142, 0.358974) (111111, 0.320513)
78 (100014, 0.208955) (111111, 0.089552)
128 (101001, 0.216667) (111111, 0.2)
8 (1003, 0.2) (111111, 0.066667)
89 (21142, 0.634731) (111111, 0.60479)
92 (1, 0.266667) (111111, 0.066667)
61 (31122, 0.463415) (111111, 0.422764)
197 (11113, 0.513274) (111111, 0.238938)
195 (100002, 0.416

# Define features

In [128]:
q_len = []
for q in qidInfo.values():
    if len(q[2].split(" ")) >4:
        print(q)
    q_len.append(len(q[2].split(" ")))
print(max(q_len))


print(qidInfo[99])


(2, 'faceted', 'french lick resort and casino')
(44, 'ambiguous', 'map of the united states')
(54, 'faceted', 'president of the united states')
(59, 'faceted', 'how to build a fence')
(70, 'faceted', 'to be or not to be that is the question')
(101, 'faceted', 'ritz carlton lake las vegas')
(118, 'faceted', 'poem in your pocket day')
(133, 'faceted', 'all men are created equal')
(149, 'faceted', 'uplift at yellowstone national park')
(169, 'faceted', 'battles in the civil war')
(177, 'faceted', 'best long term care insurance')
10
(99, 'ambiguous', 'satellite')


In [21]:
def parseLine(line):
    data = line.replace("\n","").split("\t")
    qid = int(data[0])
    df_list = json.loads(data[2])
    docid = int(data[4])
    docno = data[5]
    doclen = int(data[6])
    tf_list = np.array(json.loads(data[7]))
    return df_list, tf_list, doclen

In [83]:
def getTermFeatures(qid_str):
    directory = "/home/muntean/terrier-passage/tfs-per-qid/all-matches-top-10000/"
    filename = directory + "all-matches-fields-tfs-qid-" + qid_str + "-filtered.txt"
    
    countLines = 0
    with open(filename, "r") as inputFile:    
        df_list, sum_tf_list, sum_doclen = parseLine(inputFile.readline())
        countLines += 1
        #print(df_list, sum_tf_list, sum_doclen)
        for line in inputFile:
            countLines += 1
            df_list, tf_list, doclen = parseLine(line)
            #print(df_list, tf_list, doclen)
            sum_tf_list = sum_tf_list + tf_list
            sum_doclen = sum_doclen + doclen
  
        avg_tf_list =  sum_tf_list / countLines
        avg_doclen = sum_doclen / countLines
        
    return df_list, np.sum(avg_tf_list, axis=0), avg_doclen, avg_tf_list

In [95]:
getTermFeatures("177")

([2181380, 3281536, 4558633, 8101661, 10630912],
 array([  7.6783,   6.0622,  10.0557,   6.9605,   9.1605,  15.1481]),
 949.8307,
 array([[ 4.0618,  2.9265,  5.0182,  3.4823,  4.406 ,  7.0163],
        [ 1.1869,  0.9437,  1.5259,  1.0338,  1.3868,  2.1968],
        [ 1.3799,  1.2612,  2.0393,  1.4368,  1.9702,  3.6685],
        [ 0.9174,  0.7555,  1.181 ,  0.8117,  1.1203,  1.9064],
        [ 0.1323,  0.1753,  0.2913,  0.1959,  0.2772,  0.3601]]))

In [24]:
def idf(N, df):
    return math.log((N - df + 0.5) / (df + 0.5))

# GET them features

In [125]:
"""
We have 2 query types: faceted, ambiguous
"""
N = 50220423
AVG_LEN = 963.90334

def getQueryFeatures(info):
    featureDict=defaultdict(float)
    
    # Query Type
    if info[1] == "faceted":
        featureDict["faceted"]=1
        featureDict["ambiguous"]=0
    else:
        featureDict["faceted"]=0
        featureDict["ambiguous"]=1
        
    # Query len
    featureDict["qlen"] = len(info[2].split(" "))
    
    # Stopwords
    featureDict["num_stopwords"] = len([x for x in info[2].split(" ") if x in ENGLISH_STOP_WORDS])
    
    # Term info
    df_list, tf_list_avg, doclen_avg, tf_list_per_term  = getTermFeatures(str(qid))
    
    featureDict["deltaQlenDF"] = len(info[2].split(" ")) - len(df_list) # should be #stopwords but ...
    
#     dfs = np.zeros(5)  # max 5 terms so we 0 fill the vector
#     for j in range(len(df_list)):
#         dfs[j]=df_list[j]
#     for i, df in enumerate(dfs):
#         featureDict["DF"+str(i+1)] = df
           
#     for i, df in enumerate(dfs):
#         featureDict["IDF"+str(i+1)] = idf(N, df)
        
#     term_tf_list = np.zeros(5*6) # max 5 terms each with a list of 6 TFs_avg
#     for i, tf_list in enumerate(tf_list_per_term):
#         for j, value in enumerate(tf_list):
#             term_tf_list[(i*6)+j] = value
#     for i, term_tf in enumerate(term_tf_list):
#         featureDict["T"+str(i//6+1)+"TF" + str(i%6 + 1)] = term_tf
    
    
    featureDict["maxDF"] = max(df_list)
    featureDict["minDF"] = min(df_list)
    featureDict["avgDF"] = np.mean(df_list)
    
#     featureDict["maxIDF"] = max([idf(N, df) for df in df_list])
#     featureDict["minIDF"] = min([idf(N, df) for df in df_list])
#     featureDict["avgIDF"] = np.mean([idf(N, df) for df in df_list])
    
#     for i,tf_avg in enumerate(tf_list_avg):
#         featureDict["avgTF"+str(i+1)] = tf_avg
#     featureDict["avgDoclen"] = doclen_avg
    
#     deltaTF = [t - s for s, t in zip(tf_list_avg, tf_list_avg[1:])]
#     for i, delta in enumerate(deltaTF):
#         featureDict["deltaTF"+str(i+1)] = delta

    return featureDict

# Regression Setup

In [126]:
X_as_list = list()
y = list()
for qid, best in query2best.items():
    featureDict = getQueryFeatures(qidInfo[qid])
    X_as_list.append(list(featureDict.values()))
    fieldWeight = str(best[0]).zfill(6)
    y.append(int(fieldWeight[0]))

feature_name = list(featureDict.keys())

# X is the matrix
# y are the classes
X = np.array(X_as_list)
y = np.array(y)

In [127]:
print(X)
print(y)
print(len(y))
print(len(feature_name))
print(feature_name)

[[  0.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   4.26870000e+04
    4.26870000e+04   4.26870000e+04]
 [  1.00000000e+00   0.00000000e+00   3.00000000e+00 ...,   4.86093000e+05
    4.79420000e+04   2.11310667e+05]
 [  1.00000000e+00   0.00000000e+00   2.00000000e+00 ...,   7.17545000e+05
    6.43767000e+05   6.80656000e+05]
 ..., 
 [  1.00000000e+00   0.00000000e+00   3.00000000e+00 ...,   8.14602200e+06
    1.03621600e+06   3.42953200e+06]
 [  1.00000000e+00   0.00000000e+00   4.00000000e+00 ...,   1.13052770e+07
    1.02399000e+05   5.46757425e+06]
 [  1.00000000e+00   0.00000000e+00   3.00000000e+00 ...,   1.06634160e+07
    5.15455500e+06   7.65860067e+06]]
[2 1 1 5 0 1 0 1 0 2 1 1 0 1 1 1 2 2 1 1 0 0 0 0 0 1 1 0 0 1 4 1 5 0 1 0 2
 2 0 1 1 3 1 2 1 0 1 1 2 0 0 3 0 1 3 0 2 1 0 1 2 4 2 1 0 0 1 1 3 1 0 1 5 1
 1 0 2 1 0 0 1 1 2 0 0 0 0 1 4 0 1 3 0 0 0 2 2 0 1 4 2 0 0 0 0 0 2 1 2 2 4
 1 1 0 0 4 1 0 1 1 3 3 0 0 2 1 2 1 0 2 1 3 0 0 0 0 0 2 5 2 0 2 1 0 0 1 2 1
 0 0 0 4 4 2 1 1 5 0

In [96]:
# Standardization
# http://scikit-learn.org/stable/modules/preprocessing.html
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler() # [0,1]
max_abs_scaler = preprocessing.MaxAbsScaler() # [-1,1]

# Regression attempts

In [97]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

regressor1 = DecisionTreeRegressor(random_state=0, max_depth=2)
regressor2 = DecisionTreeRegressor(random_state=0, max_depth=5)

## K-Fold

In [123]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

k_fold = KFold(n_splits=5)
mse_avg=[]
acc_avg=[]
for train, test in k_fold.split(X):
    
    max_abs_scaler.fit(X[train])  # Don't cheat - fit only on training data
    xtrain_s = max_abs_scaler.transform(X[train])
    xtest_s = max_abs_scaler.transform(X[test])  # apply same transformation to test data
    
    regressor1.fit(xtrain_s, y[train])
    y_pred = regressor1.predict(xtest_s)
    y_pred_floor = y_pred.astype(int)
    y_pred_round = np.around(y_pred)
    print(y[test])
    print(y_pred)
    print(y_pred_floor)
    print(y_pred_round)
    print("MSE round: ", mean_squared_error(y[test], y_pred_round))
    print("Accuracy round: ", sum([x==y for (x,y) in zip(y[test], y_pred_round)])/len(y_pred))
    print("MSE : ",mean_squared_error(y[test], y_pred_floor))
    mse_avg.append(mean_squared_error(y[test], y_pred_floor))
    print("Accuracy : ", sum([x==y for (x,y) in zip(y[test], y_pred_floor)])/len(y_pred))
    acc_avg.append(sum([x==y for (x,y) in zip(y[test], y_pred_floor)])/len(y_pred))
    print()
    
print("AVG MSE: ", np.mean(mse_avg))
print("AVG Accuracy: ", np.mean(acc_avg))
    


[2 1 1 5 0 1 0 1 0 2 1 1 0 1 1 1 2 2 1 1 0 0 0 0 0 1 1 0 0 1 4 1 5 0 1 0 2
 2 0 1]
[ 2.3125      1.33018868  2.3125      1.33018868  1.33018868  1.33018868
  0.6         1.33018868  0.6         1.33018868  0.6         1.33018868
  1.33018868  1.33018868  1.33018868  1.33018868  1.33018868  1.33018868
  2.3125      0.6         2.3125      0.6         1.33018868  1.33018868
  1.33018868  1.33018868  0.6         0.6         1.33018868  2.3125
  1.33018868  1.33018868  0.6         1.33018868  1.33018868  1.33018868
  1.33018868  1.33018868  1.33018868  1.33018868]
[2 1 2 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 2 0 2 0 1 1 1 1 0 0 1 2 1 1 0 1 1 1 1
 1 1 1]
[ 2.  1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  2.  1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  2.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]
MSE round:  1.65
Accuracy round:  0.375
MSE :  1.85
Accuracy :  0.4

[1 3 1 2 1 0 1 1 2 0 0 3 0 1 3 0 2 1 0 1 2 4 2 1 0 0 1 1 3 1 0 1 5 1 1 0 2
 1 0 0]
[ 1.34108527  1.34108527  1.341085

In [136]:
###
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

# scaler = preprocessing.StandardScaler().fit(X)
sgdreg = SGDRegressor(penalty='l2', alpha=0.15, n_iter=200, random_state=5)

mse_avg=[]
acc_avg=[]
for train, test in k_fold.split(X):
    scaler = StandardScaler()
    scaler.fit(X[train])  # Don't cheat - fit only on training data
    xtrain_s = scaler.transform(X[train])
    xtest_s = scaler.transform(X[test])  # apply same transformation to test data
    sgdreg.fit(xtrain_s,y[train])
    y_pred = sgdreg.predict(xtest_s).astype(int)
    print(y[test])
    print(y_pred)
    print(sgdreg.predict(xtest_s))
    print("MSE : ", mean_squared_error(y[test], y_pred))
    mse_avg.append(mean_squared_error(y[test], y_pred))
    print("Accuracy : ", sum([x==y for (x,y) in zip(y[test], y_pred)])/len(y_pred))
    acc_avg.append(sum([x==y for (x,y) in zip(y[test], y_pred)])/len(y_pred))
print()
print("AVG MSE: ", np.mean(mse_avg))
print("AVG Accuracy: ", np.mean(acc_avg))
    

[2 1 1 5 0 1 0 1 0 2 1 1 0 1 1 1 2 2 1 1 0 0 0 0 0 1 1 0 0 1 4 1 5 0 1 0 2
 2 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1]
[ 1.29707661  1.15695515  1.26423829  1.36219072  1.14699484  1.17599891
  1.14499764  1.34095126  1.28143778  1.40516753  1.21050363  1.29604342
  1.38602418  1.55127647  1.37257414  1.23420657  1.2966385   1.31155975
  1.29675436  1.16243875  1.29681091  1.39437792  1.20828107  1.29544582
  1.36234894  1.31661951  1.3945324   1.39432985  1.25258457  1.2124705
  1.26182283  0.95701276  1.27503533  1.23543683  1.39091466  1.55935745
  1.39355091  1.41904123  1.28664983  1.29136069]
MSE :  1.55
Accuracy :  0.4
[1 3 1 2 1 0 1 1 2 0 0 3 0 1 3 0 2 1 0 1 2 4 2 1 0 0 1 1 3 1 0 1 5 1 1 0 2
 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1
 1 1 1]
[ 1.15818469  1.20696874  1.32037871  1.09168528  1.32384138  1.32077966
  1.31635627  1.29391781  1.11498027  1.29279272  1.01551173  1.32187563
  1.034

/home/muntean/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/muntean/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/muntean/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/muntean/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarni

In [130]:
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = GradientBoostingRegressor(**params)

mse_avg=[]
acc_avg=[]
for train, test in k_fold.split(X):
    
    max_abs_scaler.fit(X[train])  # Don't cheat - fit only on training data
    xtrain_s = max_abs_scaler.transform(X[train])
    xtest_s = max_abs_scaler.transform(X[test])  # apply same transformation to test data
    
    clf.fit(xtrain_s, y[train])
    y_pred = clf.predict(xtest_s)
    y_pred_floor = y_pred.astype(int)
    y_pred_round = np.around(y_pred)
    print(y[test])
    print(y_pred)
    print(y_pred_floor)
    print(y_pred_round)
    print("MSE round: ", mean_squared_error(y[test], y_pred_round))
    print("Accuracy round: ", sum([x==y for (x,y) in zip(y[test], y_pred_round)])/len(y_pred))
    print("MSE : ",mean_squared_error(y[test], y_pred_floor))
    mse_avg.append(mean_squared_error(y[test], y_pred_floor))
    print("Accuracy : ", sum([x==y for (x,y) in zip(y[test], y_pred_floor)])/len(y_pred))
    acc_avg.append(sum([x==y for (x,y) in zip(y[test], y_pred_floor)])/len(y_pred))
    print()
    
print("AVG MSE: ", np.mean(mse_avg))
print("AVG Accuracy: ", np.mean(acc_avg))


[2 1 1 5 0 1 0 1 0 2 1 1 0 1 1 1 2 2 1 1 0 0 0 0 0 1 1 0 0 1 4 1 5 0 1 0 2
 2 0 1]
[ 1.81004271  1.54570222  0.77956171  1.16474611  1.10067929  0.64642318
  0.64266835  0.40875506  0.57046098  0.88992458  0.53888626  1.60137233
  0.75427298  2.72878582  0.41888383  1.53299837  0.77011574  1.86142833
  1.60408665  0.6299007   1.60408665  0.56719069  1.08373668  0.7139055
  0.67752099  1.24845118  0.67486576  0.71353429  1.82102998  2.09552515
  1.54063256  1.37077095  0.95213756  0.70278094  2.88471194  2.18323179
  1.1106651   1.58774636  0.73021718  1.21531581]
[1 1 0 1 1 0 0 0 0 0 0 1 0 2 0 1 0 1 1 0 1 0 1 0 0 1 0 0 1 2 1 1 0 0 2 2 1
 1 0 1]
[ 2.  2.  1.  1.  1.  1.  1.  0.  1.  1.  1.  2.  1.  3.  0.  2.  1.  2.
  2.  1.  2.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  1.  1.  1.  3.  2.
  1.  2.  1.  1.]
MSE round:  1.925
Accuracy round:  0.275
MSE :  2.0
Accuracy :  0.4

[1 3 1 2 1 0 1 1 2 0 0 3 0 1 3 0 2 1 0 1 2 4 2 1 0 0 1 1 3 1 0 1 5 1 1 0 2
 1 0 0]
[ 0.88396451  1.0813034   0.644